# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "BPXDI1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [8]:
# enter your code here

model = sm.GEE.from_formula('BPXDI1 ~ 1', groups = 'group', cov_struct = sm.cov_struct.Exchangeable(), data = da)
result = model.fit()
print(result.summary())
print(result.cov_struct.summary())

                               GEE Regression Results                              
Dep. Variable:                      BPXDI1   No. Observations:                 5102
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                 106
                      Estimating Equations   Max. cluster size:                 226
Family:                           Gaussian   Mean cluster size:               170.1
Dependence structure:         Exchangeable   Num. iterations:                     5
Date:                     Mon, 24 Feb 2020   Scale:                         162.315
Covariance type:                    robust   Time:                         14:48:14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     70.0088      0.419    167.219      0.000      69.188      70.829
Skew:  

__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [35]:
# enter your code here

da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

model = sm.GEE.from_formula('BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI', groups = 'group', cov_struct = sm.cov_struct.Exchangeable(), data = da)
result = model.fit()
print(result.summary())
print(result.cov_struct.summary())

                               GEE Regression Results                              
Dep. Variable:                      BPXDI1   No. Observations:                 5102
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                 106
                      Estimating Equations   Max. cluster size:                 226
Family:                           Gaussian   Mean cluster size:               170.1
Dependence structure:         Exchangeable   Num. iterations:                     7
Date:                     Mon, 24 Feb 2020   Scale:                         158.611
Covariance type:                    robust   Time:                         15:05:09
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            65.3013      1.235     52.856      0.000      62.88

__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [33]:
# enter your code here

male = da[da['RIAGENDR'] == 1]
female = da[da['RIAGENDR'] == 2]


model_male = sm.GEE.from_formula('BPXDI1 ~ RIDAGEYR + BMXBMI', groups = 'group', 
                            cov_struct = sm.cov_struct.Exchangeable(), data = male)
result_male = model_male.fit()
print(result_male.summary())

model_female = sm.GEE.from_formula('BPXDI1 ~ RIDAGEYR + BMXBMI', groups = 'group', 
                            cov_struct = sm.cov_struct.Exchangeable(), data = female)
result_female = model_female.fit()
print(result_female.summary())

                               GEE Regression Results                              
Dep. Variable:                      BPXDI1   No. Observations:                 2462
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                  38
                      Estimating Equations   Max. cluster size:                 111
Family:                           Gaussian   Mean cluster size:                82.1
Dependence structure:         Exchangeable   Num. iterations:                     7
Date:                     Mon, 24 Feb 2020   Scale:                         173.641
Covariance type:                    robust   Time:                         15:02:59
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     66.8949      1.751     38.210      0.000      63.464      70.326
RIDAGEY

__Q3a.__ What do you learn by comparing these two fitted models?

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [53]:
# enter your code here

da["DMDEDUC2x"] = da.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})

da['age_cen'] = da.groupby('group')['RIDAGEYR'].apply(lambda x : x - x.mean())

da = da.dropna()

model = sm.MixedLM.from_formula('BPXDI1 ~ age_cen + RIAGENDRx + BMXBMI + DMDEDUC2x', groups = 'group', 
                            re_formula = "0 + age_cen", data = da)
result = model.fit()
result.summary()

/opt/conda/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
===================================================================
Model:                MixedLM    Dependent Variable:    BPXDI1     
No. Observations:     5100       Method:                REML       
No. Groups:           30         Scale:                 157.7605   
Min. group size:      105        Likelihood:            -20155.7785
Max. group size:      226        Converged:             Yes        
Mean group size:      170.0                                        
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                64.097    0.832 77.029 0.000 62.466 65.727
RIAGENDRx[T.Male]         2.819    0.355  7.952 0.000  2.124  3.514
DMDEDUC2x[T.HS]          -1.481    0.519 -2.853 0.004 -2.498 -0.464
DMDEDUC2x[T.SomeCollege] -0.632    0.482 -1.310 0.190 -1.578  0.313
DMDEDUC2x[T.lt9]         -1.339    0.631 -2.124 0.034 -2.575 -0.103
DMDEDUC2x[T.x9_11]       -0.528    0.626 -0.843 0.399 -1.754  0.699
age_cen                  -0.033    0.015 -2.189 0.029 -0.063 -0.003
BMXBMI                    0.175    0.025  6.865 0.000  0.125  0.225
age_cen Var               0.004    0.000                           
===================================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

__Q4b:__ Include a random intercept for age, and describe your findings.